In [13]:
# Ensure the model is in evaluation mode
model.eval()

# Assuming your model has parameters you want to analyze
params = model.parameters()

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a simplified GraphSAGE model for demonstration
class GraphSAGE(nn.Module):
    def __init__(self, num_nodes, num_features, embedding_dim):
        super(GraphSAGE, self).__init__()
        self.embedding = nn.Embedding(num_nodes, embedding_dim)  # Initial node feature embedding layer
        # Add more layers as needed for your GraphSAGE model
        self.fc1 = nn.Linear(embedding_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)  # Assuming num_classes for classification

    def forward(self, nodes):
        # Assuming nodes are node indices or feature tensors
        embedded_nodes = self.embedding(nodes)
        x = F.relu(self.fc1(embedded_nodes))
        x = F.relu(self.fc2(x))
        output = self.fc3(x)
        return output

# Example: Instantiate your GraphSAGE model
num_nodes = 1000
num_features = 10
embedding_dim = 64
num_classes = 2

model = GraphSAGE(num_nodes, num_features, embedding_dim)

# Assuming your model is trained and you want to analyze the embedding layer weights
params = model.parameters()

# Analyze the weights of the embedding layer to determine node feature importance
node_feature_importance = {}

for name, param in model.named_parameters():
    if 'embedding.weight' in name:  # Assuming the embedding layer is named 'embedding'
        # Extracting the weights and possibly biases if applicable
        weights = param.data
        # Example: Compute feature importance based on L2 norms of weights
        feature_importance = torch.norm(weights, p=2, dim=1)  # L2 norm across embedding dimensions
        # Assuming node indices correspond to feature importance
        for node_idx in range(num_nodes):
            node_feature_importance[node_idx] = feature_importance[node_idx].item()

# Example: Print or use node_feature_importance as needed
print(node_feature_importance)


{0: 7.984978199005127, 1: 9.270102500915527, 2: 8.328527450561523, 3: 7.2966628074646, 4: 7.689210891723633, 5: 7.507038593292236, 6: 8.146912574768066, 7: 7.975327014923096, 8: 7.825663089752197, 9: 7.971288681030273, 10: 8.048027038574219, 11: 7.731367588043213, 12: 8.807099342346191, 13: 7.5691938400268555, 14: 7.256919860839844, 15: 8.78984260559082, 16: 8.678597450256348, 17: 6.58143949508667, 18: 8.485867500305176, 19: 7.021364212036133, 20: 8.038467407226562, 21: 7.753540992736816, 22: 7.401472091674805, 23: 7.55379581451416, 24: 7.243206024169922, 25: 8.94741153717041, 26: 7.73679780960083, 27: 7.395263671875, 28: 8.341538429260254, 29: 8.179967880249023, 30: 8.421466827392578, 31: 7.261127948760986, 32: 8.734519958496094, 33: 7.5467305183410645, 34: 9.026705741882324, 35: 7.398048400878906, 36: 7.992112159729004, 37: 7.1195387840271, 38: 7.009881496429443, 39: 8.111169815063477, 40: 7.890989780426025, 41: 8.10085678100586, 42: 8.21937084197998, 43: 6.88549280166626, 44: 7.2991